# Build a RAG system with Llama 3B-Instruct for your PDFs

In this quick tutorial, we'll build a simple RAG system with an LLM from Meta AI - Llama 3, specifically the `Llama-3-8B-Instruct` version that you can get on the Hugging Face Hub.
We'll use [Unstructured Serverless API](https://unstructured.io/) for preprocessing PDF files, LangChain for RAG, FAISS for vector storage, and HuggingFace `transformers` to get the model. Let's go!

Install all the libraries, and get your [Unstructured API key](https://unstructured.io/api-key-hosted) - it comes with a 14-day trial, and a cap of 1000 pages/day.

In [1]:
!pip install -qU "unstructured-ingest[pdf]" unstructured langchain langchain-community transformers accelerate bitsandbytes sentence-transformers faiss-gpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 21.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 939.2 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 7.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 79.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 56.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 52.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 117.5 MB/s eta 0:00:00
 

In [2]:
import os

os.environ["UNSTRUCTURED_API_KEY"] = "" # Add your key here
os.environ["UNSTRUCTURED_URL"] ="" # You can find the URL in your personalized dashboard

In [3]:
from unstructured_ingest.v2.pipeline.pipeline import Pipeline
from unstructured_ingest.v2.interfaces import ProcessorConfig
from unstructured_ingest.v2.processes.connectors.local import (
    LocalIndexerConfig,
    LocalDownloaderConfig,
    LocalConnectionConfig,
    LocalUploaderConfig
)
from unstructured_ingest.v2.processes.partitioner import PartitionerConfig
from unstructured_ingest.v2.processes.chunker import ChunkerConfig

We will use the ingest functionality to partition PDF files in a local directory.

In [4]:
directory_with_pdfs="/content/data"
directory_with_results="/content/output"

Pipeline.from_configs(
    context=ProcessorConfig(),
    indexer_config=LocalIndexerConfig(input_path=directory_with_pdfs),
    downloader_config=LocalDownloaderConfig(),
    source_connection_config=LocalConnectionConfig(),
    partitioner_config=PartitionerConfig(
        partition_by_api=True,
        api_key=os.getenv("UNSTRUCTURED_API_KEY"),
        partition_endpoint=os.getenv("UNSTRUCTURED_API_URL"),
        strategy="hi_res",
        additional_partition_args={
            "split_pdf_page": True,
            "split_pdf_concurrency_level": 15,
            },
        ),
    uploader_config=LocalUploaderConfig(output_dir=directory_with_results)
).run()


2024-10-19 17:00:22,339 MainProcess INFO     created index with configs: {"input_path": "/content/data", "recursive": false}, connection configs: {"access_config": "**********"}
2024-10-19 17:00:22,343 MainProcess INFO     Created download with configs: {"download_dir": null}, connection configs: {"access_config": "**********"}
2024-10-19 17:00:22,345 MainProcess INFO     created partition with configs: {"strategy": "hi_res", "ocr_languages": null, "encoding": null, "additional_partition_args": {"split_pdf_page": true, "split_pdf_concurrency_level": 15}, "skip_infer_table_types": null, "fields_include": ["element_id", "text", "type", "metadata", "embeddings"], "flatten_metadata": false, "metadata_exclude": [], "metadata_include": [], "partition_endpoint": null, "partition_by_api": true, "api_key": "*******", "hi_res_model_name": null}
2024-10-19 17:00:22,348 MainProcess INFO     Created upload with configs: {"output_dir": "/content/output"}, connection configs: {"access_config": "*****

Load document elements from json outputs, create LangChain documents from document chunks and their metadata, and ingest those documents into the FAISS vectorstore.

Set up the retriever.

In [5]:
from unstructured.staging.base import elements_from_json

def load_processed_files(directory_path):
    elements = []
    for filename in os.listdir(directory_path):
        if filename.endswith('.json'):
            file_path = os.path.join(directory_path, filename)
            try:
                elements.extend(elements_from_json(filename=file_path))
            except IOError:
                print(f"Error: Could not read file {filename}.")

    return elements

elements = load_processed_files(directory_with_results)

In [6]:
from langchain_core.documents import Document
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings

documents = []
for element in elements:
    metadata = element.metadata.to_dict()
    documents.append(Document(page_content=element.text, metadata=metadata))

db = FAISS.from_documents(documents, HuggingFaceEmbeddings(model_name="BAAI/bge-base-en-v1.5"))
retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": 4})

<ipython-input-6-bdc6496cbd8e>:10: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  db = FAISS.from_documents(documents, HuggingFaceEmbeddings(model_name="BAAI/bge-base-en-v1.5"))
/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
INFO: Use pytorch device_name: cuda
INFO: Load pretrained SentenceTransformer: BAAI/bge-base-en-v1.5
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The se

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

INFO: Loading faiss with AVX2 support.
INFO: Could not load library with AVX2 support due to:
ModuleNotFoundError("No module named 'faiss.swigfaiss_avx2'")
INFO: Loading faiss.
INFO: Successfully loaded faiss.


In [12]:
print(documents)

[Document(metadata={'filetype': 'application/pdf', 'languages': ['eng'], 'page_number': 1, 'filename': 'tcas-test.pdf', 'data_source': {'record_locator': {'path': '/content/data/tcas-test.pdf'}, 'date_modified': '1729357143.4431336', 'date_processed': '1729357222.4066236', 'permissions_data': [{'mode': 33188}]}}, page_content='(z uree SVOl) 2952 LRUBLLUILERL[T }@ELMSMU‘IHELHM&LMN ﬂliUL"(‘! Lﬂl:USIQEUmt!!d.zt!‘l‘llgl.%LHLHSlQEU"EMSLUL‘SBIUBE‘I @PSLU’!L»WU‘I@HE‘ISEH Le'), Document(metadata={'text_as_html': '<table><tbody><tr><td>31</td><td>LVAUI</td><td></td></tr><tr><td>LZLLELE หลักสูตร/</td><td>(IQL!LUI’LUI’L) %e&amp;wmwﬂm LZLLRELE UL</td><td>7Y 8 menenLe</td></tr></tbody></table>', 'filetype': 'application/pdf', 'languages': ['eng'], 'page_number': 1, 'filename': 'tcas-test.pdf', 'data_source': {'record_locator': {'path': '/content/data/tcas-test.pdf'}, 'date_modified': '1729357143.4431336', 'date_processed': '1729357222.4066236', 'permissions_data': [{'mode': 33188}]}}, page_content=

Now, let's finally set up llama 3 to use for text generation in the RAG system.

This is a gated model, which means you first need to go to the [model's page](https://huggingface.co/meta-llama/Meta-Llama-3-8B-Instruct), log in, review terms and conditions, and request access to it. To use the model in the notebook, you need to log in with your Hugging Face token (get it in your profile's settings).

In [11]:
from huggingface_hub.hf_api import HfFolder

HfFolder.save_token("")

To run this tutorial in the free Colab GPU, we'll need to quantize the model:

In [10]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_name = "meta-llama/Meta-Llama-3-8B-Instruct"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True, bnb_4bit_use_double_quant=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=bnb_config)
tokenizer = AutoTokenizer.from_pretrained(model_name)

OSError: You are trying to access a gated repo.
Make sure to have access to it at https://huggingface.co/meta-llama/Meta-Llama-3-8B-Instruct.
403 Client Error. (Request ID: Root=1-6713e81f-46d2431763845bad3e3ab859;3d369670-f567-4740-9e6f-8d9e740ad3c7)

Cannot access gated repo for url https://huggingface.co/meta-llama/Meta-Llama-3-8B-Instruct/resolve/main/config.json.
Access to model meta-llama/Meta-Llama-3-8B-Instruct is restricted and you are not in the authorized list. Visit https://huggingface.co/meta-llama/Meta-Llama-3-8B-Instruct to ask for access.

Set up Llama 3 and a simple RAG chain.
Make sure to follow the prompt format for best results:

```
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

{{ system_prompt }}<|eot_id|><|start_header_id|>user<|end_header_id|>

{{ user_msg_1 }}<|eot_id|><|start_header_id|>assistant<|end_header_id|>

{{ model_answer_1 }}<|eot_id|>
```

In [ ]:
from langchain.llms import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from transformers import pipeline
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

text_generation_pipeline = pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    temperature=0.2,
    do_sample=True,
    repetition_penalty=1.1,
    return_full_text=False,
    max_new_tokens=200,
    eos_token_id=terminators,
)

llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

prompt_template = """
<|start_header_id|>user<|end_header_id|>
You are an assistant for answering questions about IPM.
You are given the extracted parts of a long document and a question. Provide a conversational answer.
If you don't know the answer, just say "I do not know." Don't make up an answer.
Question: {question}
Context: {context}<|eot_id|><|start_header_id|>assistant<|end_header_id|>
"""

prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=prompt_template,
)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 0.3. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFacePipeline`.
  warn_deprecated(


Tada! Your RAG is ready to use. Pass a question, the retriver will add relevant context from your document, and Llama3 will generate an answer.
Here, my document was a chapter from a book on IPM that stands for "Integrated Pest Management".  

In [ ]:
question = "What is considered a cultural control in IPM?"
rag_chain.invoke(question)

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


'Based on the context provided, I can tell you that cultural controls are one of the key components of Integrated Pest Management (IPM). In fact, according to the text, cultural controls refer to "procedural modifications that reduce the food, water, harborage, and access used by pests."\n\nIn other words, cultural controls involve making changes to the environment or ecosystem to prevent pests from thriving or reproducing. This might include practices such as adjusting irrigation schedules, pruning plants to improve air circulation, or removing weeds that provide shelter for pests.\n\nBy incorporating these types of measures into your IPM strategy, you can reduce the reliance on chemical pesticides and create a more sustainable approach to managing pests. Does that make sense?'